In [13]:
import os
import json
import random

result_dir = 'result'


def dump_jsonl(data, file_path):
    with open(file_path, 'w') as f:
        for line in data:
            f.write(json.dumps(line, ensure_ascii=False) + '\n')

def generate_function_pure(func_name):
    opcode = random.choice(['+', '-', '*', '/'])
    oprand = random.randint(1, 20)
    func = f"""
def {func_name}(x):
    return x {opcode} {oprand}

"""
    
    return func

def execute_function(func_text, func_name, x):
    exec(func_text)
    y = eval(f"{func_name}({x})")
    
    return y


In [14]:
def generate_one_data_pure(num_func=10):
    index_ptr = 0
    
    functions = dict()
    concate_function = ""
    for _ in range(num_func):
        index_ptr += random.randint(1, 3)
        function_name = f"func_{index_ptr}"
        func_text = generate_function_pure(function_name)
        functions[function_name] = func_text
        concate_function += func_text
    
    return functions, concate_function

def generate_pure_code_runs(num):
    data = []
    for _ in range(num):
        functions, concate_function = generate_one_data_pure(num_func=random.randint(1, 15))
        eval_func = random.choice(list(functions.keys()))
        eval_oprand = random.randint(1, 20)
        correct_answer = execute_function(functions[eval_func], eval_func, eval_oprand)
        
        # Generate incorrect answers
        incorrect_answers = set()
        while len(incorrect_answers) < 3:
            option = correct_answer + random.randint(-10, 10)
            if option != correct_answer:
                incorrect_answers.add(option)
        
        # Create options and ensure correct answer is in one of them
        options = list(incorrect_answers)
        correct_index = random.randint(0, 3)
        options.insert(correct_index, correct_answer)
        
        # Map options to A, B, C, D
        choices = ['A', 'B', 'C', 'D']
        answer_letter = choices[correct_index]
        
        question = (
            f"There is a function called {eval_func} in the following Python code:\n\n"
            f"{concate_function}\n\n"
            f"Compute the exact value of {eval_func}({eval_oprand})."
        )
        
        data.append({
            "question": question,
            "A": str(options[0]),
            "B": str(options[1]),
            "C": str(options[2]),
            "D": str(options[3]),
            "correct_answer": answer_letter
        })
    
    return data


In [15]:
pure_data = generate_pure_code_runs(50)

In [16]:
# dump_jsonl(result_data, os.path.join(result_dir, 'code_run_dataset.jsonl'))
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
    
dump_jsonl(pure_data, os.path.join(result_dir, 'code_run.jsonl'))